<a href="https://colab.research.google.com/github/Ximena5745/Developer/blob/main/Indicadores_Kawak.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from google.colab import files

In [2]:
# Cargar archivo desde botón
uploaded = files.upload()

Saving consulta_indicadores (3).xlsx to consulta_indicadores (3).xlsx


In [3]:
# Obtener el nombre del archivo cargado
filename = list(uploaded.keys())[0]

In [8]:
# Obtener el nombre del archivo cargado
filename = list(uploaded.keys())[0]

# Leer contenido del archivo Excel
Reporte_Indicadores = pd.read_excel(filename)

# Vista previa del DataFrame
print("Vista previa del archivo cargado:")
print(Reporte_Indicadores.head())


Vista previa del archivo cargado:
    ID                                         nombre       clasificacion  \
0   77       Disponibilidad de servicios tecnológicos  Estrat&eacute;gico   
1  121  Ejecución presupuestal de la nómina acumulada           Operativo   
2  124         Indice de Frecuencia de accidentalidad           Operativo   
3  125                            Indice de severidad           Operativo   
4  128                    Ausentísmo por causa médica           Operativo   

    sentido                                        proceso  \
0  Positivo  GESTIÓN DE LA TECNOLOGÍA E INNOVACIÓN DIGITAL   
1  Negativo        GESTIÓN Y DESARROLLO DEL TALENTO HUMANO   
2  Negativo     GESTIÓN DE SEGURIDAD Y SALUD EN EL TRABAJO   
3  Negativo     GESTIÓN DE SEGURIDAD Y SALUD EN EL TRABAJO   
4  Negativo     GESTIÓN DE SEGURIDAD Y SALUD EN EL TRABAJO   

                                         descripcion  \
0  Relación disponibilidad servicio infraestructu...   
1  Mide la eficien

In [12]:
# Filtrar campo 'Fecha' y eliminar registros vacíos
Reporte_Indicadores = Reporte_Indicadores.dropna(subset=['fecha'])

In [13]:
# Reemplazar '.' por ',' en la columna 'resultado'
if 'resultado' in Reporte_Indicadores.columns:
    Reporte_Indicadores['resultado'] = Reporte_Indicadores['resultado'].astype(str).str.replace('.', ',')

In [16]:
# Filtrar registros como 'Serie Unica'
Serie_Unica = Reporte_Indicadores[Reporte_Indicadores['Tipo'] == 'Serie Unica']

# Filtrar registros como 'Multiserie'
Multiserie = Reporte_Indicadores[Reporte_Indicadores['Tipo'] == 'Multiserie']

# Guardar los nuevos DataFrames en archivos Excel separados
Serie_Unica.to_excel('Serie_Unica.xlsx', index=False)
Multiserie.to_excel('Multiserie.xlsx', index=False)

print("Nuevo archivo guardado: Reporte_Indicadores_Filtrado.xlsx")
print("Archivo guardado: Serie_Unica.xlsx")
print("Archivo guardado: Multiserie.xlsx")


Nuevo archivo guardado: Reporte_Indicadores_Filtrado.xlsx
Archivo guardado: Serie_Unica.xlsx
Archivo guardado: Multiserie.xlsx


In [15]:
print(Reporte_Indicadores.head())


    ID                                         nombre       clasificacion  \
0   77       Disponibilidad de servicios tecnológicos  Estrat&eacute;gico   
1  121  Ejecución presupuestal de la nómina acumulada           Operativo   
2  124         Indice de Frecuencia de accidentalidad           Operativo   
3  125                            Indice de severidad           Operativo   
4  128                    Ausentísmo por causa médica           Operativo   

    sentido                                        proceso  \
0  Positivo  GESTIÓN DE LA TECNOLOGÍA E INNOVACIÓN DIGITAL   
1  Negativo        GESTIÓN Y DESARROLLO DEL TALENTO HUMANO   
2  Negativo     GESTIÓN DE SEGURIDAD Y SALUD EN EL TRABAJO   
3  Negativo     GESTIÓN DE SEGURIDAD Y SALUD EN EL TRABAJO   
4  Negativo     GESTIÓN DE SEGURIDAD Y SALUD EN EL TRABAJO   

                                         descripcion  \
0  Relación disponibilidad servicio infraestructu...   
1  Mide la eficiencia presupuestal de la nómina i...